In [ ]:
import numpy as np
import numpy.lib.recfunctions
import scipy
import scipy.interpolate
import pandas
import platypus.io.logs

In [ ]:
# Import the data from the specified logfile
log_filename = '../logs/platypus_20161126_150201.txt'
data = platypus.io.logs.load(log_filename)

# Define useful access variables.
pose = data['pose']
position = pose[['latitude', 'longitude']]

# Print the available sensors and channels for this logfile.
print "Available sensors/channels:"
for s in data.keys():
    if s == 'pose' or s == 'BATTERY':
        continue
    for c in data[s].dtypes.keys():
        print "  {:s}, {:s}".format(s, str(c))

In [ ]:
# Select the sensor and the name of the channel for that sensor.
sensor_name = 'ATLAS_DO'
sensor_data = 'do'

In [ ]:
# Extract the pose timing and the sensor data of interest.
pose_times = pose.index.values.astype(np.float64)

sensor = data[sensor_name]
sensor_times = sensor.index.values.astype(np.float64)

# Linearly interpolate the position of the sensor at every sample.
sensor_pose_interpolator = scipy.interpolate.interp1d(pose_times, pose[['latitude', 'longitude']],
                                                      axis=0, bounds_error=False)

# Add the position information back to the sensor data.
sensor = sensor.join(pandas.DataFrame(sensor_pose_interpolator(sensor_times), sensor.index,
                                      columns=('latitude', 'longitude')))

# Remove columns that have NaN values (no pose information).
sensor_valid = np.all(np.isfinite(sensor), axis=1)
sensor = sensor[sensor_valid]

In [ ]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

In [ ]:
# Create a map centered on this data log.
center = [pose['latitude'].median(), pose['longitude'].median()]
zoom = 17
m = Map(center=center, zoom=zoom)
m

In [ ]:
# Add trail for the vehicle to map
pl = Polyline(locations=position.as_matrix().tolist())
pl.fill_opacity = 0.0
pl.weight = 2
m += pl

In [ ]:
# Add a data overlay for the map
data_padding = 0.0001  # degrees lat/lon
data_bounds = [(position.min() - data_padding).tolist(),
               (position.max() + data_padding).tolist()]

data_grid = xv, yv = meshgrid(x, y, sparse=False, indexing='ij')

io = ImageOverlay(url='http://ipython.org/_static/IPy_header.png', bounds=data_bounds)
m += io

In [ ]:
m.bounds